## Evaluierung
Evaluierung des Modells.


### Model

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd

df = pd.read_csv("bmw-dataset/bmw.csv", sep=',',  engine='python')

def kategorisieren(modell):
    if isinstance(modell, str):  
        modell = modell.strip() 
        if modell in ["X1", "X2", "X3", "X4", "X5", "X6", "X7"]:
            return "SUV"
        elif modell in ["2 Series", "4 Series", "6 Series", "8 Series"]:
            return "Coupé"
        elif modell in ["Z3", "Z4"]:
            return "Roadster"
        elif modell in ["i3", "i8"]:
            return "Elektro"
        elif modell in ["M2", "M3", "M4", "M5", "M6"]:
            return "Sportwagen"
        elif modell in ["1 Series"]:
            return "Kompaktklasse"
        elif modell == "3 Series":
            return "Mittelklasse"
        elif modell == "5 Series":
            return "Obere Mittelklasse"
        elif modell in ["7 Series", "8 Series"]:
            return "Luxus"
        else:
            return "Unbekannt"
    else:
        return "Unbekannt"


df["Kategorie"] = df["model"].apply(kategorisieren)


df.to_csv("autos_mit_kategorie.csv", index=False)

df = df[(df['price'] > 1000) & (df['price'] < 80000)]

df_encoded = pd.get_dummies(df, columns=['model', 'transmission', 'fuelType', 'Kategorie'])

X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"MSE (Random Forest): {mse:.2f}")

mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MSE (Random Forest): 6087562.53
MAE: 1564.44


## Felder
Als Erstes bestimme ich, welche Felder meines Datensatzes am aussagekräftigsten sind. 
Dies setze ich um, indem ich immer wieder eine andere Spalte entferne und das Modell neu trainiere.
Um den Unterschied darzustellen, benutze ich den MAE da dieser Einfacher zu interpretieren ist. Er stellt den durchschnittlichen absoluten Unterschied zwischen den vorhergesagten Preisen und den tatsächlichen Preisen.


### Entfernen tax Spalte
Minimale Änderung, ca. 15 Euro mehr differenz. Heisst hat nicht so einen grossen Einfluss auf die vorhersage.


In [24]:
X_train_reduced = X_train.drop('tax', axis=1)
X_test_reduced = X_test.drop('tax', axis=1)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_reduced, y_train)

y_pred = model.predict(X_test_reduced)

mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 1580.32


### Entfernen mileage Spalte
Die Mileage hat schon einen grösseren Einfluss auf die Vorhersage, hier ist die Differenz zum Normalen MAE 250 Euro mehr. 

In [25]:
X_train_reduced = X_train.drop('mileage', axis=1)
X_test_reduced = X_test.drop('mileage', axis=1)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_reduced, y_train)

y_pred = model.predict(X_test_reduced)

mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 1808.60


### Entfernung EngineSize Spalte
Enginze Size hat wiederum keinen grossen Einfluss mit nur ca. 30 Euro Unterschied.

In [26]:
X_train_reduced = X_train.drop('engineSize', axis=1)
X_test_reduced = X_test.drop('engineSize', axis=1)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_reduced, y_train)

y_pred = model.predict(X_test_reduced)

mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 1596.52


### Entfernung mpg Spalte
Die Spalte mpg (miles per gallon) hat einen sehr grossen Einfluss auf die Daten, was mich verwundert. Mit einer Differenz von fast 350 Euro. Vielleicht waren die Käufer der Autos ja sehr umweltbewusst.

In [27]:
X_train_reduced = X_train.drop('mpg', axis=1)
X_test_reduced = X_test.drop('mpg', axis=1)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_reduced, y_train)

y_pred = model.predict(X_test_reduced)

mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 1894.62


### Warheitsmatrix
Um eine Wahrheitsmatrix zu erstellen, brauche ich eine Bedingung, welche überprüft werden kann. Da ich den Preis vorhersage, muss ich irgendeine Bedingung für diesen festlegen. Ich habe mich dazu entschieden, dass ein Auto über 35'000 als teuer gilt und alles darunter als gutes Angebot. 

In [35]:
from sklearn.metrics import confusion_matrix

condition = 35000

y_test_class = (y_test >= condition).astype(int)
y_pred_class = (y_pred >= condition).astype(int)

cm = confusion_matrix(y_test_class, y_pred_class)

true_negatives  = cm[0, 0]
false_positives = cm[0, 1] 
false_negatives = cm[1, 0] 
true_positives  = cm[1, 1] 

recall = true_positives / (true_positives + false_negatives)
precision = true_positives / (true_positives + false_positives)


print(f"Trefferquote: {recall:.2f}")
print(f"Genaugikeit: {precision:.2f}")

Trefferquote: 0.81
Genaugikeit: 0.87


## Fazit
Zuerst dachte ich das mein Modell sehr schlecht ist, da der MSE sehr hoch war. Als ich jedoch andere Werte berechnet habe wie den MAE oder die Differenz von einzelnen Preisen und Preisvorhersagen habe ich gemerkt, dass es gar nicht so schlecht ist. Eine Differenz von 1500 Euro ist akzeptabel. Es gibt auch Werte, die weit darunter liegen, im dreistelligen Bereich. Jedoch gibt es auch Ausreisser, die bis zu 5000 Euro daneben liegen. Ich denke ein Problem bei meinem Modell ist der Datensatz, gewisse Werte darin haben gar keinen Einfluss auf den Preis, während andere Werte den 10 oder 20 Fachen Einfluss haben. Grundsätzlich funktioniert das Modell, nicht vertrauenswürdig, aber es funktioniert. 